In [1]:
import ads
import os
import pandas as pd

In [2]:
def get_ads_query_limits():
    # There is a limited number of queries allowed per ADS API token.
    # This function makes a dummy query asking for this information
    r = ads.RateLimits('SearchQuery')
    q = ads.SearchQuery(q='machine-learning')
    print(r.limits)

In [3]:
def reformat_list(item,joinchar=" "):
    # Where there are lists of items, return elements separated by "joinchar"
    itemout = item
    if item == None:
        itemout = ""
    if isinstance(item,list):
        itemout = joinchar.join(item)
    return itemout

def reformat_list_doi(item):
    # Get only the first of the DOIs listed
    return item.split(" ")[0]

def impact_factor(journal):
    # For now this information is coded here, but it should be read from a data source (DB or list)
    ifs={"Astronomy and Astrophysics":["6,24","12/69","Q1","D2"],
        "The Astrophysical Journal":["8,811","8/69","Q1","D2"],
        "The Astronomical Journal":["5,491","15/69","Q1","D2"],
        "Monthly Notices of the Royal Astronomical Society":["5,235","17/69","Q1","D3"],
        "The Astrophysical Journal Supplement Series":["14,7","7/91","Q1","D1"],
        "Journal of Fluid Mechanics":["6,8","35/590","Q1","D1"], 
        "Infrared Physics and Technology":["5,0","120/415","Q2","D3"],
        "Scientific Reports":["6,9","11/120","Q1","D1"]}
    out = ["-","-","-","-"]
    if journal in ifs.keys():
        out = ifs[journal]
    return out

def get_author_query_for_IPARCOS(author, year, outfile="out.csv"):
    
    fout = open(outfile,'w')
    query = ads.SearchQuery(author=author, year=year, doctype="article", sort="date", \
                            fl=['id', 'title', 'author', 'pub', 'issn', 'year', \
                                'volume', 'issue', 'eid', 'doi',"esources", "bibcode"])
    for paper in query:
        paper.title = reformat_list(paper.title)
        paper.author = reformat_list([p.replace(',','') for p in paper.author],joinchar=", ")
        paper.issn = reformat_list(paper.issn)
        paper.doi = reformat_list(paper.doi)
        paper.pub = reformat_list(paper.pub)
        paper.issn = reformat_list(paper.issn)
        paper.year = reformat_list(paper.year)
        paper.volume = reformat_list(paper.volume)
        paper.issue = reformat_list(paper.issue)
        paper.eid = reformat_list(paper.eid)
        paper.doi = reformat_list_doi(paper.doi)
        
        if ("VizieR" in paper.pub) or ("arXiv" in paper.pub):
            continue

        ifactor = reformat_list(impact_factor(paper.pub)).replace(' ','","')
        
        lineout = '"{0}","{1}","{2}","{3}","{4}","{5}","{6}","{7}","{8}","","","SCOPUS","{9}","{10}","{11}"'.format( \
              paper.title, \
              paper.author, \
              author, \
              paper.pub, \
              paper.issn, \
              paper.year, \
              paper.volume, \
              paper.issue, \
              paper.eid, \
              ifactor, \
              paper.doi, \
              "http://doi.org/"+paper.doi)
        print(lineout)
        fout.write(lineout+'\n')
    fout.close()

In [70]:
get_ads_query_limits()

{'limit': '5000', 'remaining': '4994', 'reset': '1677664807'}


In [71]:
get_author_query_for_IPARCOS("Montenegro-Montes, F.M.",2022,outfile="Montenegro-Montes.csv")

"SEPIA345: A 345 GHz dual polarization heterodyne receiver channel for SEPIA at the APEX telescope","Meledin D., Lapkin I., Fredrixon M., Sundin E., Ferm S. -E., Pavolotsky A., Strandberg M., Desmaris V., López C., Bergman P., Olberg M., Conway J., Torstensson K., Durán C., Montenegro-Montes F. M., De Breuck C., Belitsky V.","Montenegro-Montes, F.M.","Astronomy and Astrophysics","0004-6361","2022","668","","A2","","","SCOPUS","6,24","12/69","Q1","D2","10.1051/0004-6361/202244211","http://doi.org/10.1051/0004-6361/202244211"
"Extremely high spectral resolution measurements of the 450 µm atmospheric window at Chajnantor with APEX","Pardo J. R., De Breuck C., Muders D., González J., Montenegro-Montes F. M., Pérez-Beaupuits J. P., Cernicharo J., Prigent C., Serabyn E., Mroczkowski T., Phillips N.","Montenegro-Montes, F.M.","Astronomy and Astrophysics","0004-6361","2022","664","","A153","","","SCOPUS","6,24","12/69","Q1","D2","10.1051/0004-6361/202243409","http://doi.org/10.1051/0004-6361/2

In [72]:
# Read the csv into a pandas dataframe and remove any possible duplications that one could have (e.g., if you have merged into a csv various queries)
df = pd.read_csv("Montenegro-Montes.csv", header=None,names = ["Title","AuthorList", "Author", "Publication", "ISSN", "Year", "Volume", "Issue", "EID", "A", "B","ImpactFactor-source", "ImpactFactor", "Ranking", "QuartileCode", "DecileCode", "DOI", "DOIURL"])
df = df[~df.index.duplicated(keep='first')]
df.drop(columns=["A","B"],inplace=True)

In [73]:
df

,Title,AuthorList,Author,Publication,ISSN,Year,Volume,Issue,EID,ImpactFactor-source,ImpactFactor,Ranking,QuartileCode,DecileCode,DOI,DOIURL
0,SEPIA345: A 345 GHz dual polarization heterody...,"Meledin D., Lapkin I., Fredrixon M., Sundin E....","Montenegro-Montes, F.M.",Astronomy and Astrophysics,0004-6361,2022,668,NaN,A2,SCOPUS,"6,24",12/69,Q1,D2,10.1051/0004-6361/202244211,http://doi.org/10.1051/0004-6361/202244211
1,Extremely high spectral resolution measurement...,"Pardo J. R., De Breuck C., Muders D., González...","Montenegro-Montes, F.M.",Astronomy and Astrophysics,0004-6361,2022,664,NaN,A153,SCOPUS,"6,24",12/69,Q1,D2,10.1051/0004-6361/202243409,http://doi.org/10.1051/0004-6361/202243409
2,First Sagittarius A* Event Horizon Telescope R...,"Event Horizon Telescope Collaboration, Akiyama...","Montenegro-Montes, F.M.",The Astrophysical Journal,0004-637X,2022,930,2.0,L12,SCOPUS,"8,811",8/69,Q1,D2,10.3847/2041-8213/ac6674,http://doi.org/10.3847/2041-8213/ac6674
3,First Sagittarius A* Event Horizon Telescope R...,"Event Horizon Telescope Collaboration, Akiyama...","Montenegro-Montes, F.M.",The Astrophysical Journal,0004-637X,2022,930,2.0,L13,SCOPUS,"8,811",8/69,Q1,D2,10.3847/2041-8213/ac6675,http://doi.org/10.3847/2041-8213/ac6675
